In [1]:
!pip install gdown nltk tqdm

In [2]:
!gdown '1mNalWH4TZFhAvNEIv_RcWQS3RuvNSo52'
!gdown '1_hHef3PtVl8QGv2J1JgIpA726NhBv5GE'
!gdown '1XxKLEOq4XsMshcDzz2ZMAQhrMhohN0av'

Downloading...
From: https://drive.google.com/uc?id=1mNalWH4TZFhAvNEIv_RcWQS3RuvNSo52
To: /content/dev_data.csv
100% 7.39M/7.39M [00:00<00:00, 72.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_hHef3PtVl8QGv2J1JgIpA726NhBv5GE
To: /content/test_data.csv
100% 15.1M/15.1M [00:01<00:00, 14.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XxKLEOq4XsMshcDzz2ZMAQhrMhohN0av
To: /content/train_data.csv
100% 53.2M/53.2M [00:00<00:00, 103MB/s]


In [3]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.WARNING)

# Importing necessary libraries
import os
import sys
import json
import logging
import math
import copy
import re
import torch
from torch import nn


from multiprocessing import Pool, cpu_count

from nltk.tokenize import sent_tokenize as nltk_sent_tokenize
import nltk
from tqdm.auto import tqdm

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd
import ast

train_data = pd.read_csv("/content/train_data.csv")
test_data = pd.read_csv("/content/test_data.csv")
dev_data = pd.read_csv("/content/dev_data.csv")

train_data = train_data.drop('evidence_top5', axis=1)
dev_data = dev_data.drop('evidence_top5', axis=1)
test_data = test_data.drop('evidence_top5', axis=1)


def convert_to_list(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return value

train_data['splited_sentences'] = train_data['splited_sentences'].apply(convert_to_list)
train_data['Evidence_List'] = train_data['Evidence_List'].apply(convert_to_list)
test_data['splited_sentences'] = test_data['splited_sentences'].apply(convert_to_list)
test_data['Evidence_List'] = test_data['Evidence_List'].apply(convert_to_list)
dev_data['splited_sentences'] = dev_data['splited_sentences'].apply(convert_to_list)
dev_data['Evidence_List'] = dev_data['Evidence_List'].apply(convert_to_list)

In [5]:
# from sklearn.model_selection import train_test_split
# from ast import literal_eval

# train_data = pd.read_csv("/content/train_final.csv", converters={"Evidence_List": lambda x: x.strip("[]").strip("''").split("', '"),"splited_sentences": lambda x: x.strip("[]").strip("''").split("', '")})
# dev_data = pd.read_csv("/content/dev_final.csv", converters={"Evidence_List": lambda x: x.strip("[]").strip("''").split("', '"),"splited_sentences": lambda x: x.strip("[]").strip("''").split("', '")})
# test_data = pd.read_csv("/content/dev_final.csv", converters={"Evidence_List": lambda x: x.strip("[]").strip("''").split("', '"),"splited_sentences": lambda x: x.strip("[]").strip("''").split("', '")})

In [6]:
train_data.head(5)

,Unnamed: 0,index,Statement,Context,annotation_id,Topic,Author,Url,labels,Evidence,splited_sentences,Evidence_List,len_evidence
0,0,3049,"Phó Thủ tướng Trần Hồng Hà thay mặt Chính phủ,...","(Chinhphu.vn) - Đây là mong muốn, gửi gắm của ...",18933775,Chính trị,Chính Phủ,https://baochinhphu.vn/lan-toa-nhung-gia-tri-v...,0,"Thay mặt Chính phủ, Thủ tướng Chính phủ, Phó T...","[(Chinhphu.vn) - Đây là mong muốn, gửi gắm của...","[Thay mặt Chính phủ, Thủ tướng Chính phủ, Phó ...",1
1,1,6811,Hành vi của Tô Văn Hải là cho phép người khác ...,"Ngày 24/3, Cơ quan Cảnh sát điều tra Công an t...",19402113,PHÁP LUẬT,Tin Tức,https://baotintuc.vn/an-ninh-trat-tu/bat-tam-g...,0,Tô Văn Hải đã có hành vi cho phép người khác đ...,"[Ngày 24/3, Cơ quan Cảnh sát điều tra Công an ...",[Tô Văn Hải đã có hành vi cho phép người khác ...,2
2,2,7270,SAWACO thông báo tạm ngưng cung cấp nước để th...,(PLO)- Theo Tổng Công ty Cấp nước Sài Gòn (SAW...,19807472,ĐÔ THỊ,Báo Pháp Luật HCM,https://plo.vn/9-quan-huyen-o-tphcm-se-bi-cup-...,1,SAWACO thông báo tạm ngưng cung cấp nước để th...,[(PLO)- Theo Tổng Công ty Cấp nước Sài Gòn (SA...,[SAWACO thông báo tạm ngưng cung cấp nước để t...,2
3,3,7423,"CLB luôn chuẩn bị rất kỹ lưỡng, chỉn chu chươn...","Với khoảng 200 thành viên, UEF Warm Hugs Club ...",19725173,Giáo dục,Người lao động,https://nld.com.vn/giao-duc-khoa-hoc/hanh-trin...,2,Là chương trình lớn nhất của CLB trong năm nên...,"[Với khoảng 200 thành viên, UEF Warm Hugs Club...",[Là chương trình lớn nhất của CLB trong năm nê...,1
4,4,6632,"ILA tiếp nhận và hỗ trợ học sinh miễn phí, Bé ...","Đi qua 2 năm dịch bệnh đầy thử thách, nhưng ch...",19021291,Giáo dục,Thanh Niên,https://thanhnien.vn/giao-duc-anh-ngu-phat-tri...,2,"Bé được tham gia kiểm tra trình độ đầu vào, tư...","[Đi qua 2 năm dịch bệnh đầy thử thách, nhưng c...","[Bé được tham gia kiểm tra trình độ đầu vào, t...",1


## BM25

In [7]:
class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = 0
        self.avgdl = 0
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer

        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for document in corpus:
            self.doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word]+=1
                except KeyError:
                    nd[word] = 1

            self.corpus_size += 1

        self.avgdl = num_doc / self.corpus_size
        return nd

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5):
        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        scores = self.get_scores(query)
        min_score = np.min(scores)
        max_score = np.max(scores)

        # Scale scores to 0-1 range
        if max_score != min_score:
            scaled_scores = (scores - min_score) / (max_score - min_score)
        else:
            scaled_scores = np.ones(self.corpus_size)

        top_n_indices = np.argsort(scaled_scores)[::-1][:n]
        top_n_scaled_scores = [scaled_scores[i] for i in top_n_indices]

        return [documents[i] for i in top_n_indices], top_n_scaled_scores


class BM25Okapi(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score.tolist()

In [8]:
def preprocess_text(text: str) -> str:
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

def evidence_top_n(splited_sentences, query, top_n):
    # sentences = split_text(context)
    tokenized_sentences = [str(doc).split(" ") for doc in splited_sentences]
    bm25 = BM25Okapi(context)
    tokenized_query = query.split(" ")
    top_docs, top_scores = bm25.get_top_n(tokenized_query, splited_sentences, top_n)

    return top_docs, top_scores

In [9]:
test_data.head(5)

,Unnamed: 0,index,Statement,Context,annotation_id,Topic,Author,Url,labels,Evidence,splited_sentences,Evidence_List,len_evidence
0,0,1383,Bộ xương bản sao được đưa đến Lodon bằng cách ...,Bản sao bộ xương hóa thạch của một trong những...,19050703,THẾ GIỚI,Tin Tức,https://baotintuc.vn/the-gioi/trung-bay-ban-sa...,1,đại diện Bảo tàng Lịch sử tự nhiên London cho ...,[Bản sao bộ xương hóa thạch của một trong nhữn...,[đại diện Bảo tàng Lịch sử tự nhiên London cho...,1
1,1,7117,"Phần vỏ bánh được làm rất công phu, từ giai đo...","Bánh cuốn Thụy Khuê là quán ăn đã qua hai đời,...",18655554,Du lịch,VnExpress,https://vnexpress.net/quan-banh-cuon-trong-ngo...,2,Gạo tẻ ngon được xay nhuyễn bằng cối đá từ xưa...,[Bánh cuốn Thụy Khuê là quán ăn đã qua hai đời...,[Gạo tẻ ngon được xay nhuyễn bằng cối đá từ xư...,6
2,2,3399,Đối tượng cố gắng bỏ chạy sau khi lực lượng cô...,(PLO)- Khi lực lượng công an đón dừng xe để ki...,18855494,AN NINH TRẬT TỰ,Báo Pháp Luật HCM,https://plo.vn/thanh-nien-tang-tru-ma-tuy-chon...,0,"Khi lực lượng công an đón dừng xe để kiểm tra,...",[(PLO)- Khi lực lượng công an đón dừng xe để k...,[Khi lực lượng công an đón dừng xe để kiểm tra...,1
3,3,559,Vì lượng mưa khổng lồ do cơn bão tạo ra đã dẫn...,Bão Freddy tàn phá đông nam châu Phi đã di chu...,19639539,Khoa học,VnExpress,https://vnexpress.net/bao-freddy-co-the-lap-ky...,2,Cơn bão trút lượng mưa khổng lồ xuống đất liền...,[Bão Freddy tàn phá đông nam châu Phi đã di ch...,[Cơn bão trút lượng mưa khổng lồ xuống đất liề...,1
4,4,6895,"Tại một địa điểm trên địa bàn huyện Củ Chi, TP...",(Chinhphu.vn) - Dù có phải là hành vi ép buộc ...,18121201,Xã hội,Chính Phủ,https://baochinhphu.vn/vu-be-trai-nghi-bi-ep-h...,0,N. được xác định là mẹ ruột của bé Th. B. chín...,[(Chinhphu.vn) - Dù có phải là hành vi ép buộc...,[N. được xác định là mẹ ruột của bé Th. B. chí...,1


In [10]:
list_evidence_top5 = []


for i in tqdm(range(len(test_data))):
    statement = test_data.Statement[i]
    context = test_data.splited_sentences[i]

    evidence_top5, top5_consine = evidence_top_n(context, statement, 5) # top 5

    list_evidence_top5.append(evidence_top5)

test_data['evidence_top5'] = list_evidence_top5

  0%|          | 0/1447 [00:00<?, ?it/s]

In [14]:
def acc_at_k(retrieved, gold, k):
    top_k_retrieved = retrieved[:k]

    remaining_gold = list(gold)
    remaining_retrieved = list(top_k_retrieved)

    correct_retrievals = 0

    gold_indices_to_remove = set()
    retrieved_indices_to_remove = set()

    for gold_idx, gold_evidence in enumerate(remaining_gold):
        for retrieved_idx, retrieved_evidence in enumerate(remaining_retrieved):
            if gold_evidence in retrieved_evidence or retrieved_evidence in gold_evidence:
                correct_retrievals += 1
                gold_indices_to_remove.add(gold_idx)
                retrieved_indices_to_remove.add(retrieved_idx)
                break

    remaining_gold = [e for i, e in enumerate(remaining_gold) if i not in gold_indices_to_remove]
    remaining_retrieved = [e for i, e in enumerate(remaining_retrieved) if i not in retrieved_indices_to_remove]

    acc_k = correct_retrievals / len(gold) if gold else 0

    return acc_k

i = 12
retrieved_evidence = test_data.evidence_top5[i]
gold_evidence = test_data.Evidence_List[i]

k = 5
acc_k_result = acc_at_k(retrieved_evidence, gold_evidence, k)
print(f"ACC@{k}: {acc_k_result}")

ACC@5: 1.0


In [15]:
def map_at_k(retrieved, gold, k):
    top_k_retrieved = retrieved[:k]

    # Initialize variables for calculation
    num_relevant = 0
    precision_at_i_sum = 0.0

    # Track which gold items have been retrieved
    retrieved_gold_indices = set()

    for i, retrieved_evidence in enumerate(top_k_retrieved):
        for gold_idx, gold_evidence in enumerate(gold):
            if gold_idx in retrieved_gold_indices:
                continue

            if gold_evidence in retrieved_evidence or retrieved_evidence in gold_evidence:
                num_relevant += 1
                retrieved_gold_indices.add(gold_idx)
                precision_at_i_sum += num_relevant / (i + 1)
                break

    # Calculate mAP@K
    map_k = precision_at_i_sum / len(gold) if gold else 0

    return map_k

# Example usage
retrieved = ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']
gold = ['doc1', 'doc3', 'doc5']
k = 5

map_k = map_at_k(retrieved, gold, k)
print(f"mAP@{k}: {map_k}")


mAP@5: 0.7555555555555555


## ACC@K

In [16]:
import numpy as np

results = {
    "all": {"all": [], 0: [], 1: [], 2: []},
    "multi": {"all": [], 0: [], 1: [], 2: []},
    "single": {"all": [], 0: [], 1: [], 2: []}
}

for i in range(len(test_data)):
    retrieved_evidence = test_data.evidence_top5[i]
    gold_evidence = test_data.Evidence_List[i]
    label = test_data.labels[i]

    acc_k_result = acc_at_k(retrieved_evidence, gold_evidence, 5)

    results["all"]["all"].append(acc_k_result)
    results["all"][label].append(acc_k_result)

    if len(gold_evidence) > 1:
        results["multi"]["all"].append(acc_k_result)
        results["multi"][label].append(acc_k_result)
    elif len(gold_evidence) == 1:
        results["single"]["all"].append(acc_k_result)
        results["single"][label].append(acc_k_result)

def calculate_mean(values):
    return round(np.mean(values), 4) if values else 0

for case in ["all", "multi", "single"]:
    print(f"acc_k_result {case}:")
    print(f"overall: {calculate_mean(results[case]['all'])}")
    for label in [0, 1, 2]:
        print(f"{label}: {calculate_mean(results[case][label])}")
    print()

acc_k_result all:
overall: 0.2593
0: 0.238
1: 0.2349
2: 0.3064

acc_k_result multi:
overall: 0.2314
0: 0.2536
1: 0.1984
2: 0.2456

acc_k_result single:
overall: 0.2712
0: 0.2296
1: 0.2541
2: 0.3218



In [36]:
i = 124
retrieved_evidence = test_data.evidence_top5[i]
gold_evidence = test_data.Evidence_List[i]
print(gold_evidence)
print("--"*80)
retrieved_evidence

['Còn đối với lúa mì, tại miền Bắc, lúa mì Úc được giao dịch ở mức 8.100 đồng/kg.', 'Mặc dù giá giao dịch tại thị trường nội địa đã tăng nhẹ trong 2 ngày gần đây']
----------------------------------------------------------------------------------------------------------------------------------------------------------------


['Sở Giao dịch Hàng hóa Việt Nam (MXV)',
 'Mặc dù giá giao dịch tại thị trường nội địa đã tăng nhẹ trong 2 ngày gần đây, tuy nhiên giá nguyên liệu nhập khẩu vẫn đang thấp hơn khoảng 200 -250 đồng/kg so với hồi đầu tháng 03, đây là tín hiệu tích cực đối với các doanh nghiệp chăn nuôi và sản xuất thức ăn chăn nuôi trong nước khi mà giá nguyên liệu đầu vào hạ nhiệt',
 'Đóng cửa, có đến 26 trên tổng số 31 mặt hàng tăng giá đã kéo chỉ số MXV-Index tăng mạnh 1,4% lên 2.261 điểm, cao nhất trong 9 phiên trở lại đây',
 'Giá trị giao dịch toàn Sở đạt mức 4.000 tỷ đồng',
 'Bảng giá năng lượng kết thúc ngày giao dịch 273']

In [37]:
map_at_k(retrieved_evidence, gold_evidence, 5)

0.25

## mAP@K

In [38]:
import numpy as np

results = {
    "all": {"all": [], 0: [], 1: [], 2: []},
    "multi": {"all": [], 0: [], 1: [], 2: []},
    "single": {"all": [], 0: [], 1: [], 2: []}
}

for i in range(len(test_data)):
    retrieved_evidence = test_data.evidence_top5[i]
    gold_evidence = test_data.Evidence_List[i]
    label = test_data.labels[i]

    map_k_result = map_at_k(retrieved_evidence, gold_evidence, 1)

    results["all"]["all"].append(map_k_result)
    results["all"][label].append(map_k_result)

    if len(gold_evidence) > 1:
        results["multi"]["all"].append(map_k_result)
        results["multi"][label].append(map_k_result)
    elif len(gold_evidence) == 1:
        results["single"]["all"].append(map_k_result)
        results["single"][label].append(map_k_result)

def calculate_mean(values):
    return round(np.mean(values), 4) if values else 0

for case in ["all", "multi", "single"]:
    print(f"mAP_k_result {case}:")
    print(f"overall: {calculate_mean(results[case]['all'])}")
    for label in [0, 1, 2]:
        print(f"{label}: {calculate_mean(results[case][label])}")
    print()

mAP_k_result all:
overall: 0.0735
0: 0.064
1: 0.0582
2: 0.0987

mAP_k_result multi:
overall: 0.0607
0: 0.0539
1: 0.0512
2: 0.0895

mAP_k_result single:
overall: 0.0789
0: 0.0695
1: 0.0619
2: 0.1011



## F1@K

In [43]:
def f1_at_k(retrieved, gold, k):
    top_k_retrieved = retrieved[:k]

    # Calculate Precision@K
    num_correct_retrievals = 0
    gold_indices_to_remove = set()

    for retrieved_evidence in top_k_retrieved:
        for gold_idx, gold_evidence in enumerate(gold):
            if gold_idx in gold_indices_to_remove:
                continue

            if gold_evidence in retrieved_evidence or retrieved_evidence in gold_evidence:
                num_correct_retrievals += 1
                gold_indices_to_remove.add(gold_idx)
                break

    precision_k = num_correct_retrievals / k if k > 0 else 0

    # Calculate Recall@K
    recall_k = num_correct_retrievals / len(gold) if gold else 0

    # Calculate F1@K
    if precision_k + recall_k == 0:
        f1_k = 0
    else:
        f1_k = 2 * (precision_k * recall_k) / (precision_k + recall_k)

    return f1_k

# Example usage
retrieved = ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']
gold = ['doc1', 'doc3', 'doc5']
k = 5

f1_k = f1_at_k(retrieved, gold, k)
print(f"F1@{k}: {f1_k}")

F1@5: 0.7499999999999999


In [45]:
import numpy as np

results = {
    "all": {"all": [], 0: [], 1: [], 2: []},
    "multi": {"all": [], 0: [], 1: [], 2: []},
    "single": {"all": [], 0: [], 1: [], 2: []}
}

for i in range(len(test_data)):
    retrieved_evidence = test_data.evidence_top5[i]
    gold_evidence = test_data.Evidence_List[i]
    label = test_data.labels[i]

    f1_k_result = f1_at_k(retrieved_evidence, gold_evidence, 5)

    results["all"]["all"].append(f1_k_result)
    results["all"][label].append(f1_k_result)

    if len(gold_evidence) > 1:
        results["multi"]["all"].append(f1_k_result)
        results["multi"][label].append(f1_k_result)
    elif len(gold_evidence) == 1:
        results["single"]["all"].append(f1_k_result)
        results["single"][label].append(f1_k_result)

def calculate_mean(values):
    return round(np.mean(values), 4) if values else 0

for case in ["all", "multi", "single"]:
    print(f"mAP_k_result {case}:")
    print(f"overall: {calculate_mean(results[case]['all'])}")
    for label in [0, 1, 2]:
        print(f"{label}: {calculate_mean(results[case][label])}")
    print()

mAP_k_result all:
overall: 0.1045
0: 0.1025
1: 0.0971
2: 0.114

mAP_k_result multi:
overall: 0.1376
0: 0.1512
1: 0.1208
2: 0.1406

mAP_k_result single:
overall: 0.0904
0: 0.0765
1: 0.0847
2: 0.1073

